# Norvig's `lis.py`

Contents:

* [Scheme Syntax](#Scheme-Syntax)
* [Imports and types](#Imports-and-types)
* [The Parser](#The-Parser)
  * [Exercise 0](#Exercise-0)
* [The Built-in Environment](#The-Built-in-Environment)
* [A Calculator](#A-Calculator)
* [Non-interactive execution](#Non-interactive-execution)
  * [Exercise 1](#Exercise-1)
  * [Exercise 2](#Exercise-2)
* [The REPL](#The-REPL)

## Introduction

[Peter Norvig](https://norvig.com/) of Stanford University wrote
[`lis.py`](https://github.com/norvig/pytudes/blob/main/py/lis.py):
an interpreter for a subset of the Scheme dialect of Lisp in 132 lines of readable Python code.

Why should you study `lis.py`? This is what I got out of it:

* Learning how an interpreter works gave me a deeper understanding of Python and programming languages in general—interpreted or compiled.

* The simplicity of Scheme is a master class of language design.

* `lis.py` is a beautiful example of idiomatic Python code.


![Norvig's lispy](norvigs-lispy.png)

Before looking at the Python code, let’s get a little taste of Scheme—in case you haven’t seen it (or Lisp) before.

## Scheme Syntax


Everything in Scheme is an expression.
There are no infix operators:
all expressions use prefix notation like `(+ x 13)` instead of `x + 13`.
The same prefix notation is used for function calls—e.g. `(gcd x 13)`—and
special forms—e.g. `(define x 13)`,
which we'd write as the assignment statement `x = 13` in
Python.
The notation used by Scheme and most Lisp dialects is known as _S-expression_.

Here is a simple example in Scheme:

```scheme
(define (mod m n)
    (- m (* n (// m n))))

(define (gcd m n)
    (if (= n 0)
        m
        (gcd n (mod m n))))

(display (gcd 18 45))
```

Here is the same algorithm in Python:

In [ ]:
def mod(m, n):
    return m - (m // n * n)

def gcd(m, n):
    if n == 0:
        return m
    else:
        return gcd(n, mod(m, n))

print(gcd(18, 45))

> **HINT**: Click on the cell above to select it, then hit `[CTRL][ENTER]` to execute it.

In idiomatic Python I'd use the `%` operator instead of reinventing `mod`,
and it would be more efficient to use a `while` loop instead of recursion.
But I wanted to show two function definitions,
and make the examples as similar as possible,
to help you read the Scheme code.

Scheme has no iterative control flow commands like `while` or `for`.
Iteration is done with recursion.
Note how there are no assignments in the Scheme and Python examples.
Extensive use of recursion and minimal use of assignment
are hallmarks of programming in a functional style.

Now let's review the code of the Python 3.10 version of `lis.py`.
The complete source code with tests is in the [18-with-match/lispy/py3.10/](https://github.com/fluentpython/example-code-2e/tree/master/18-with-match/lispy/py3.10/)
directory of the Github repository
[fluentpython/example-code-2e](https://github.com/fluentpython/example-code-2e).

## Imports and types

Norvig's code does not use type hints. I added them. This notebook uses Python 3.7 to run on [Binder](https://mybinder.org/), therefore the some collections types must be imported from `typing`.

> **HINT**: Click on the cell below to select it, then hit `[SHIFT][ENTER]` to execute and select the next cell. Use `[CTRL][ENTER]` to execute a cell and keep it selected. Use these commands to execute cells as you follow along.

In [ ]:
import sys
assert sys.version_info >= (3, 7), f'Expected Python ≥ 3.7; found: {sys.version}'
sys.version

In [ ]:
import math
import operator as op
from collections import ChainMap
from typing import Any, NoReturn
from typing import Union, List, MutableMapping, Optional, Iterator

Symbol = str
Atom = Union[float, int, Symbol]
Expression = Union[Atom, List]

Environment = MutableMapping[Symbol, object]

The types defined are:

`Symbol`: Just an alias for `str`.
In _lis.py_, `Symbol` is used for identifiers,
there is no string data type with operations such as slicing, splitting etc.

`Atom`: A simple syntactic element such as a number or a `Symbol`—as opposed to a composite structure made of distinct parts, like a list.

`Expression`: The building blocks of Scheme programs are expressions made of atoms and lists, possibly nested.

> **NOTE**: Norvig's second interpreter,
[`lispy.py`](https://github.com/fluentpython/example-code-2e/blob/master/18-with-match/lispy/original/lispy.py),
supports strings as a data type, as well as advanced features like
syntactic macros, continuations, and proper tail calls.
However, `lispy.py` is almost three times longer than `lis.py`—and
harder to understand.

## The Parser

Norvig's parser is 36 lines of code showcasing the power of Python applied to handling the
simple recursive syntax of S-expression—without string data,
comments, macros, and other features of standard Scheme that make parsing more complicated (these features are implemented in `lispy.py`).

In [ ]:
def parse(program: str) -> Expression:
    "Read a Scheme expression from a string."
    return read_from_tokens(tokenize(program))

def tokenize(s: str) -> List[str]:
    "Convert a string into a list of tokens."
    return s.replace('(', ' ( ').replace(')', ' ) ').split()

def read_from_tokens(tokens: List[str]) -> Expression:
    "Read an expression from a sequence of tokens."
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        exp = []
        while tokens[0] != ')':
            exp.append(read_from_tokens(tokens))
        tokens.pop(0)  # discard ')'
        return exp
    elif ')' == token:
        raise SyntaxError('unexpected )')
    else:
        return parse_atom(token)

def parse_atom(token: str) -> Atom:
    "Numbers become numbers; every other token is a symbol."
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return Symbol(token)

The main function of that group is `parse` which takes an S-expressions as a `str`
and returns an `Expression` object: an `Atom` or a `list` that may contain more atoms and nested lists.

Norvig uses a smart trick in `tokenize`:
he adds spaces before and after each parenthesis in the input and then splits it,
resulting in a list of syntactic tokens with `'('` and `')'`
as separate tokens.
This shortcut works because there is no string type in the little Scheme of _lis.py_, so every `'('` or `')'` is an expression delimiter.
The recursive parsing code is in `read_from_tokens`.
I will not explain it now because I want to focus on the other parts of the interpreter.

Below are some examples of the top-level `parse` function.

> **HINT**:  To run the code in each cell and select the next, use `[SHIFT][ENTER]`. If you get `NameError: name 'parse' is not defined`, use the menu command ***Cell > Run All Above***.

In [ ]:
parse('1.5')

In [ ]:
parse('ni!')

In [ ]:
parse('''
  (define double
    (lambda (n)
      (* n 2)))
''')

The parsing rules for this subset of Scheme are simple:

1. A token that looks like a number is parsed as a `float` or `int`.
2. Anything else that is not `'('` or `')'` is parsed as a `Symbol`—a `str` to be used as an identifier. This includes source text like `+`, `set!`, and `make-counter` that are valid identifiers in Scheme but not in Python.
3. Expressions inside `'('` and `')'` are recursively parsed as lists containing atoms or nested lists that may contain atoms and more nested lists.

Using terminology of the Python interpreter, the output of `parse` is an **AST** (Abstract Syntax Tree):
a convenient representation of the Scheme program as nested lists forming a tree-like structure,
where the outermost list is the trunk, inner lists are the branches, and atoms are the leaves.

### Exercise 0

Replace the ellipis `...` with the AST for the given S-expressions, to make the comparison `True`.
To run the code in the cell, use CTRL-ENTER. 

In [ ]:
parse('9') == ...

In [ ]:
parse('x/y') == ...

In [ ]:
parse('(+ 3 7)') == ...

In [ ]:
parse('(* c (/ 9 5))') == ...

In [ ]:
parse('(+ 32 (* (/ 9 5) c ))') == ...

## The Built-in Environment


The `standard_env()` function builds and returns an `Environment` loaded
with predefined functions, similar to Python's `__builtins__` module that is always available.

In [ ]:
def standard_env() -> Environment:
    "An environment with some Scheme standard procedures."
    env: Environment = {}
    env.update(vars(math))   # sin, cos, sqrt, pi, ...
    env.update(
        {
            '+': op.add,
            '-': op.sub,
            '*': op.mul,
            '/': op.truediv,
            '>': op.gt,
            '<': op.lt,
            '>=': op.ge,
            '<=': op.le,
            '=': op.eq,
            'abs': abs,
            'append': op.add,
            'apply': lambda proc, args: proc(*args),
            'begin': lambda *x: x[-1],
            'car': lambda x: x[0],
            'cdr': lambda x: x[1:],
            'cons': lambda x, y: [x] + y,
            'eq?': op.is_,
            'equal?': op.eq,
            'length': len,
            'list': lambda *x: list(x),
            'list?': lambda x: isinstance(x, list),
            'map': lambda *args: list(map(*args)),
            'max': max,
            'min': min,
            'not': op.not_,
            'null?': lambda x: x == [],
            'number?': lambda x: isinstance(x, (int, float)),
            'procedure?': callable,
            'round': round,
            'symbol?': lambda x: isinstance(x, Symbol),
        }
    )
    return env

The `env` mapping is loaded with:

* all functions from Python's `math` module;
* selected operators from Python's `op` module;
* simple but powerful functions built with Python's `lambda`;
* Python built-ins renamed like `callable` as `procedure?` or directly mapped like `round`.

## A Calculator

This first version of `evaluate` handles expressions using built-in functions and user-defined variables.


In [ ]:
def evaluate(x: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    if isinstance(x, str):                       # variable reference
        return env[x]
    elif not isinstance(x, list):                # constant literal
        return x
    elif x[0] == 'define':                       # (define var exp)
        (_, var, exp) = x
        env[var] = evaluate(exp, env)
    else:                                        # (proc arg...)
        proc = evaluate(x[0], env)
        args = (evaluate(exp, env) for exp in x[1:])
        return proc(*args)

In [ ]:
evaluate(parse('(* 11111 11111)'), standard_env())

In [ ]:
evaluate(parse('(* (/ 123 876) 100)'), standard_env())


## Non-interactive execution

The following functions take Scheme source code as a string and execute it.

In [ ]:
def run_lines(source: str, env: Optional[Environment] = None) -> Iterator[Any]:
    global_env: Environment = ChainMap({}, standard_env())
    if env is not None:
        global_env.update(env)
    tokens = tokenize(source)
    while tokens:
        exp = read_from_tokens(tokens)
        yield evaluate(exp, global_env)


def run(source: str, env: Optional[Environment] = None) -> Any:
    for result in run_lines(source, env):
        pass
    return result

We can't define functions yet, only variables. But `run` makes it easier to experiment:

In [ ]:
percentage = """
(define a 126)
(define b (* 6 50))
(* (/ a b) 100)
"""
run(percentage)

### Exercise 1

This is the formula to convert temperatures from Celsius to Fahrenheit:

`f = (9 / 5) * c + 32`

Here is the code to convert 20°C to °F:

In [ ]:
c_to_f = """
(define c 20)
(+ 32 (* c (/ 9 5)))
"""
run(c_to_f)

The inverse conversion function is:

`c = (f − 32) * 5 / 9`

In the code below, replace `(+ 1 2)` with the expression to convert 68°F to °C.

In [ ]:
f_to_c = """
(define f 68)
(+ 1 2)
"""
run(f_to_c)

### Exercise 2

Python's `math` module includes a `factorial` function, which is part of the environment returned by `standard_env`:

In [ ]:
run('(factorial 10)')

Scheme accepts `!` as an identifier. Your task is to make Python's `factorial` available through the `!` symbol in Scheme.

**Step 2.1** Uncomment the expression below and run it to see an error. Look at the last line of the error output. What is the error? Do you understand why that is the error?

In [ ]:
# run('(! 10)') == 3628800

**Step 2.2.** Edit the `standard_env` function above to add an entry for `!`,
mapping to Python's `math.factorial` function.

**Step 2.3.** Run the expression above to confirm that the result is `True`.


### Exercise 3

In standard Scheme, the `+` operator is variadic, i.e. it accepts any number of arguments.
With 0 arguments, `(+)` returns `0`; otherwise it returns the sum of all arguments.

**Step 3.1.** Uncomment the expression below and run it to see an error. Does the error make sense to you?

In [ ]:
# run('(= 10 (+ 1 2 3 4))')

**Step 3.2.** Edit the `standard_env` function above to re-implement `+` to make the expression above return `True`.

> **HINT 3.2.1**: below is a variadic version of Python's `sum`:


In [ ]:
def variadic_sum(*args):
    return sum(args)

variadic_sum(1, 2, 3, 4)

> **HINT 3.2.2**: the same function in a single line of Python, followed by a demonstration:

In [ ]:
f = lambda *args: sum(args)
f(1, 2, 3, 4)

## The REPL

Norvig's REPL (Read-Eval-Print-Loop) is easy to undersand but not user-friendly.
If no command-line arguments are given to _lis.py_,
the `repl()` function is invoked by `main()`—defined at the end of the module.
At the `lis.py>` prompt we must enter correct and complete expressions—if
we forget to close one parenthesis, _lis.py_ crashes.

> **NOTE**: As I studied Norvig's _lis.py_ and _lispy.py_, I started a fork named
  [`mylis`](https://github.com/fluentpython/lispy/blob/main/mylis)
  which adds some features, including a REPL that accepts partial S-expressions
  and prompts for the continuation, similar to how Python's REPL
  knows we are not finished and presents the secondary prompt `...` until
  we enter a complete expression or statement that can be evaluated.
  `mylis` also handles a few errors gracefully, but it's still easy to crash.


In [ ]:
def repl(prompt: str = 'lis.py> ') -> NoReturn:
    "A prompt-read-evaluate-print loop."
    global_env: Environment = standard_env()
    while True:
        val = evaluate(parse(input(prompt)), global_env)
        if val is not None:
            print(lispstr(val))


def lispstr(exp: object) -> str:
    "Convert a Python object back into a Lisp-readable string."
    if isinstance(exp, list):
        return '(' + ' '.join(map(lispstr, exp)) + ')'
    else:
        return str(exp)

Function `repl` calls `standard_env()` to provide built-in functions for the global environment,
then enters an infinite loop reading and parsing each input line,
evaluating it in the global environment and displaying the result—unless it's `None`.
The `global_env` may be modified by `evaluate`.

`lispstr` is the inverse function of `parse`:
given a Python object representing an expression,
`parse` returns the Scheme source code for it.
For example, given `['+', 2, 3]`, the result is `'(+ 2 3)'`.